<a href="https://colab.research.google.com/github/gulamkibria775/Data_science_project/blob/main/Data_science_project/resnet_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
from google.colab import drive
drive.mount("/content/driver")

Drive already mounted at /content/driver; to attempt to forcibly remount, call drive.mount("/content/driver", force_remount=True).


In [38]:
def identity_block(x, filters, kernel_size=3):
    # Shortcut
    shortcut = x

    # First layer
    x = layers.Conv2D(filters, kernel_size, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    # Second layer
    x = layers.Conv2D(filters, kernel_size, padding='same')(x)
    x = layers.BatchNormalization()(x)

    # Add the shortcut to the main path
    x = layers.add([x, shortcut])
    x = layers.Activation('relu')(x)

    return x


In [39]:
def convolutional_block(x, filters, kernel_size=3, strides=2):
    # Shortcut
    shortcut = x

    # First layer
    x = layers.Conv2D(filters, kernel_size, strides=strides, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    # Second layer
    x = layers.Conv2D(filters, kernel_size, padding='same')(x)
    x = layers.BatchNormalization()(x)

    # Shortcut path
    shortcut = layers.Conv2D(filters, 1, strides=strides, padding='same')(shortcut)
    shortcut = layers.BatchNormalization()(shortcut)

    # Add the shortcut to the main path
    x = layers.add([x, shortcut])
    x = layers.Activation('relu')(x)

    return x


In [40]:
def build_resnet50(input_shape, num_classes):
    inputs = layers.Input(shape=input_shape)

    # Initial convolutional layer
    x = layers.Conv2D(64, 7, strides=2, padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling2D(3, strides=2, padding='same')(x)

    # Stacking convolutional and identity blocks
    x = convolutional_block(x, 64)
    x = identity_block(x, 64)
    x = identity_block(x, 64)

    x = convolutional_block(x, 128)
    x = identity_block(x, 128)
    x = identity_block(x, 128)

    x = convolutional_block(x, 256)
    x = identity_block(x, 256)
    x = identity_block(x, 256)
    x = identity_block(x, 256)

    x = convolutional_block(x, 512)
    x = identity_block(x, 512)
    x = identity_block(x, 512)

    # Global average pooling and fully connected layer
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(num_classes, activation='softmax')(x)

    model = models.Model(inputs, x)

    return model


In [41]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Assuming you have defined the build_resnet50 function and set up your data directories

# Set up data generators
train_data_dir = '/content/driver/MyDrive/All_data_file/ResNet50-master/rooms_dataset'
# validation_data_dir = 'path/to/validation_data'
# test_data_dir = 'path/to/test_data'

input_shape = (224, 224, 3)  # Adjust the input shape according to your images
num_classes = 3  # Adjust the number of classes in your dataset

batch_size = 32

train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical'
)

# validation_generator = validation_datagen.flow_from_directory(
#     validation_data_dir,
#     target_size=input_shape[:2],
#     batch_size=batch_size,
#     class_mode='categorical'
# )

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical'
)

# Build and compile the ResNet50 model
resnet50_model = build_resnet50(input_shape, num_classes)

resnet50_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
epochs = 5

history = resnet50_model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    # validation_data=validation_generator,
    # validation_steps=validation_generator.samples // batch_size
)

# Evaluate the model on the test set
test_loss, test_acc = resnet50_model.evaluate(test_generator)
print(f'Test Accuracy: {test_acc}')

# Save the trained model if needed
resnet50_model.save('resnet50_model.h5')


Found 415 images belonging to 3 classes.
Found 57 images belonging to 3 classes.
Epoch 1/5
12/12 [==============================] - 106s 7s/step - loss: 2.2145 - accuracy: 0.3568
Epoch 2/5
12/12 [==============================] - 72s 6s/step - loss: 1.1132 - accuracy: 0.4517
Epoch 3/5
12/12 [==============================] - 67s 5s/step - loss: 1.0537 - accuracy: 0.4674
Epoch 4/5
12/12 [==============================] - 69s 6s/step - loss: 0.9633 - accuracy: 0.5170
Epoch 5/5
2/2 [==============================] - 3s 849ms/step - loss: 3.4114 - accuracy: 0.1930
Test Accuracy: 0.19298245012760162


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
